<a href="https://colab.research.google.com/github/Iddrisu-Abdulai/AnalysisR/blob/main/BookRecommendationEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
!pip install kaggle
!pip install fuzzywuzzy
import os
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import numpy as np
from fuzzywuzzy import fuzz, process
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving kaggle (1).json to kaggle (1) (1).json


{'kaggle (1) (1).json': b'{"username":"iddrisuabdulai","key":"d3b7ed1780e518f9a47284bbfd764061"}'}

In [29]:
!cat ~/.kaggle/kaggle.json

{"username":"iddrisuabdulai","key":"646c97d4134cadfa0c642cbd5165d2f9"}

In [37]:

# Move the kaggle.json file to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d syedjaferk/book-crossing-dataset
!unzip book-crossing-dataset.zip


cp: cannot stat 'kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/syedjaferk/book-crossing-dataset
License(s): CC0-1.0
 20% 5.00M/24.6M [00:00<00:00, 40.5MB/s]
100% 24.6M/24.6M [00:00<00:00, 117MB/s] 
Archive:  book-crossing-dataset.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [64]:
# Load the dataset, skipping bad lines
df_books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')
df_ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding='latin-1')

# Rename columns for clarity
df_books.columns = ['isbn', 'book_title', 'book_author', 'year_of_publication', 'publisher', 'image_url_s', 'image_url_m', 'image_url_l']
df_ratings.columns = ['user_id', 'isbn', 'rating']

# Merge the datasets on 'isbn'
df = pd.merge(df_ratings, df_books[['isbn', 'book_title']], on='isbn', how='inner')

# Display the first few rows of the merged dataset
print(df.head())

<ipython-input-64-c397ff6514f7>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv('BX-Books.csv', sep=';', encoding='latin-1', on_bad_lines='skip')


   user_id        isbn  rating  \
0   276725  034545104X       0   
1   276726  0155061224       5   
2   276727  0446520802       0   
3   276729  052165615X       3   
4   276729  0521795028       6   

                                          book_title  
0                               Flesh Tones: A Novel  
1                                   Rites of Passage  
2                                       The Notebook  
3                                     Help!: Level 1  
4  The Amsterdam Connection : Level 4 (Cambridge ...  


In [54]:
# Remove users with less than 200 ratings
user_counts = df['user_id'].value_counts()
df = df[df['user_id'].isin(user_counts[user_counts >= 200].index)]

# Remove books with less than 100 ratings
book_counts = df['isbn'].value_counts()
df = df[df['isbn'].isin(book_counts[book_counts >= 100].index)]

# Create user-item matrix
user_item_matrix = df.pivot(index='user_id', columns='isbn', values='rating').fillna(0)

# Normalize the data
user_item_matrix_norm = user_item_matrix.subtract(user_item_matrix.mean(axis=1), axis=0)

'To ensure statistical significance, remove from the dataset users with less than 200 ratings
and books with less than 100 ratings.'

These ristrictions have resulted in an empty DataFrame, and so a suggestion to reduce the these thresholds to 50 for users and 20 for books to include more data in the analysis.


In [65]:
# Remove users with less than 200 ratings
user_counts = df['user_id'].value_counts()
df = df[df['user_id'].isin(user_counts[user_counts >= 50].index)] # Reduced minimum user ratings to 50

# Remove books with less than 100 ratings
book_counts = df['isbn'].value_counts()
df = df[df['isbn'].isin(book_counts[book_counts >= 20].index)] # Reduced minimum book ratings to 20

# Create user-item matrix
user_item_matrix = df.pivot(index='user_id', columns='isbn', values='rating').fillna(0)

# Normalize the data
user_item_matrix_norm = user_item_matrix.subtract(user_item_matrix.mean(axis=1), axis=0)

In [62]:

# Check if the book title exists in the 'book_title' column
book_title = "The Queen of the Damned \(Vampire Chronicles \(Paperback\)\)"
if df_books['book_title'].str.contains(book_title).any():
    print(f"The book '{book_title}' is present in the dataset.")
else:
    print(f"The book '{book_title}' is not found in the dataset.")

The book 'The Queen of the Damned \(Vampire Chronicles \(Paperback\)\)' is present in the dataset.


In [70]:
# Implement KNN
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(user_item_matrix_norm.values.T)

# Create the get_recommends function
def get_recommends(book_title):
    # Find the closest match in the dataset based on book title
    best_match = process.extractOne(book_title, df_books['book_title'], scorer=fuzz.token_set_ratio)

    # Check if the match is above a certain threshold (e.g., 85%)
    if best_match[1] >= 85:
        matched_book_title = best_match[0]
        print(f"Found closest match: '{matched_book_title}'")

        # Get the ISBN of the matched book
        matched_isbn = df_books.loc[df_books['book_title'] == matched_book_title, 'isbn'].iloc[0]

        # Check if the ISBN exists in the user-item matrix columns
        if matched_isbn not in user_item_matrix.columns:
            print(f"Book with ISBN '{matched_isbn}' not found in the dataset for recommendations. Returning popular books instead.")
            # Return top 5 most popular books as fallback:
            popular_books = df['book_title'].value_counts().head(5).index.tolist()
            return [(book, None) for book in popular_books]
        else:
            # Get recommendations based on ISBN
            book_index = user_item_matrix.columns.get_loc(matched_isbn)
            distances, indices = model_knn.kneighbors(user_item_matrix_norm.iloc[:, book_index].values.reshape(1, -1), n_neighbors=6)

            similar_books = []
            for i in range(1, len(distances.flatten())):
                # Get the ISBN of the similar book
                similar_book_isbn = user_item_matrix.columns[indices.flatten()[i]]

                # Get the title of the similar book using the ISBN
                similar_book_title = df_books.loc[df_books['isbn'] == similar_book_isbn, 'book_title'].iloc[0]

                similar_books.append((similar_book_title, distances.flatten()[i]))

            return similar_books
    else:
        print(f"The book '{book_title}' is not found in the dataset.")
        return []

In [71]:
# Example usage
book_title = "The Queen of the Damned \(Vampire Chronicles \(Paperback\)\)"
recommendations = get_recommends(book_title)
print(recommendations)


Found closest match: 'The Queen of the Damned (Vampire Chronicles (Paperback))'
[('The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.48360643695239025), ('The Vampire Lestat (Vampire Chronicles, Book II)', 0.530769523881157), ('Memnoch the Devil (Vampire Chronicles, No 5)', 0.6613540486441493), ('Interview with the Vampire', 0.6815903489220037), ('Lasher: Lives of the Mayfair Witches (Lives of the Mayfair Witches)', 0.7966637366741259)]
